In [1]:
import glob, os, re
import pandas as pd
from obspy import UTCDateTime
import tkinter as tk
from PIL import Image, ImageTk

In [2]:
class ImageDisplay:
    def __init__(self, root, image_paths):
        self.root = root
        self.image_paths = image_paths
        self.current_image_index = 0
        self.user_responses = []  # 用于存储用户的响应

        # 加载第一张图片
        self.load_image(self.current_image_index)

        # 绑定键盘事件
        self.root.bind('<Key>', self.on_key_press)

        # 创建按钮框架
        self.button_frame = tk.Frame(root)
        self.button_frame.pack()

        # 创建 "Yes" 按钮
        self.yes_button = tk.Button(self.button_frame, text="Yes (Left)", command=self.on_yes)
        self.yes_button.pack(side=tk.LEFT, padx=10)

        # 创建 "No" 按钮
        self.no_button = tk.Button(self.button_frame, text="No (Right)", command=self.on_no)
        self.no_button.pack(side=tk.RIGHT, padx=10)

    def load_image(self, index):
        image_path = self.image_paths[index]
        try:
            image = Image.open(image_path)
            self.photo = ImageTk.PhotoImage(image)
            if hasattr(self, 'label'):
                self.label.config(image=self.photo)
            else:
                self.label = tk.Label(self.root, image=self.photo)
                self.label.pack()
        except:
            print(image_path)
            pass



    def on_yes(self):
        # print("User likes the image!")
        self.user_responses.append(True)  # 将 True 写入列表
        self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        if self.current_image_index == 0:
            self.end_loop()  # 如果回到第一张图片，结束循环
        else:
            self.load_image(self.current_image_index)

    def on_no(self):
        # print("User does not like the image!")
        self.user_responses.append(False)  # 将 False 写入列表
        self.current_image_index = (self.current_image_index + 1) % len(self.image_paths)  # 循环更新索引
        if self.current_image_index == 0:
            self.end_loop()  # 如果回到第一张图片，结束循环
        else:
            self.load_image(self.current_image_index)

    def end_loop(self):
        # print("User responses:", self.user_responses)
        print('exit ~~~~')
        self.root.quit()  # 退出主循环

    def on_key_press(self, event):
        
        if event.keysym == 'Left':  # 按下 'y' 键表示 Yes
            self.on_yes()
        elif event.keysym == 'Right':  # 按下 'n' 键表示 No
            self.on_no()
        else:
            print(f"Pressed {event.keysym}")  # 打印按键信息


In [3]:
path = f'../Results/Detection_fishcall/csv'
figpath = '../Results/Detection_fishcall/fig'

In [4]:

for csvpath in sorted(glob.glob(f'{path}/result_230430.csv')):
    df = pd.read_csv(csvpath)
    df_ = df[df['coeff']>=0.70]
    
    imagelist = []
    for i in range(len(df_)):
        begintmp = re.split('[(|)|,]',df_['time_min_date'].values[i]); endtmp = re.split('[(|)|,]',df_['time_max_date'].values[i])
        tr_begin = UTCDateTime(int(begintmp[1]),int(begintmp[2]),int(begintmp[3]),int(begintmp[4]),int(begintmp[5]),int(begintmp[6]),int(begintmp[7]))
        tr_end = UTCDateTime(int(endtmp[1]),int(endtmp[2]),int(endtmp[3]),int(endtmp[4]),int(endtmp[5]),int(endtmp[6]),int(endtmp[7]))
        
        yy = str(tr_begin.year).zfill(2)
        mm = str(tr_begin.month).zfill(2)
        dd = str(tr_begin.day).zfill(2)        
        hour = str(tr_begin.hour).zfill(2)
        minute = str(tr_begin.minute).zfill(2)
        second = str(tr_begin.second).zfill(2)
        date_time = f'{yy[2:]}{mm}{dd}_{hour}{minute}{second}'
        
        imagelist.append(f'{figpath}/{date_time}.png')
    print(len(imagelist))
    
root = tk.Tk()
root.title("Image Display")

# 图片路径列表
image_paths = imagelist 

# 创建 ImageDisplay 实例
image_display = ImageDisplay(root, image_paths)

# 运行主循环
root.mainloop()
response = image_display.user_responses
# 确保主循环结束后销毁窗口
root.destroy()

print('Saving file........... ')
tempdf = pd.DataFrame({'keep': response})
df__ = df_.reset_index(drop=True)
# df_combine = pd.concat([df__, tempdf], join='outer', axis=1) 
# df_combine.to_csv(f"../Results/Detection_fishcall/manual_csv_0.75/{csvpath.rsplit('/')[-1]}",index=False)

6


TclError: can't invoke "destroy" command: application has been destroyed

In [ ]:
imagelist

In [7]:
df_combine = pd.concat([df__, tempdf], join='outer', axis=1) 


In [8]:
df_combine

,time_min_offset,time_max_offset,frequency_min,frequency_max,duration,time_min_date,time_max_date,SPLpp,peaks,low_freq,SNR,coeff,keep
0,1.823133,1.921803,738,2583,0.09867,"UTCDateTime(2023, 4, 30, 0, 4, 11, 823133","UTCDateTime(2023, 4, 30, 0, 4, 11, 921803",141.237561,2,NaN,2.861173,0.749530,False
1,2.981433,3.157323,1914,5994,0.17589,"UTCDateTime(2023, 4, 30, 1, 2, 3, 981433","UTCDateTime(2023, 4, 30, 1, 2, 4, 157323",148.700722,2,NaN,4.870859,0.757081,False
2,0.553293,0.651963,1104,2271,0.09867,"UTCDateTime(2023, 4, 30, 10, 2, 31, 553293","UTCDateTime(2023, 4, 30, 10, 2, 31, 651963",140.211807,2,NaN,1.646015,0.723051,False
3,4.216953,4.332783,1158,2781,0.11583,"UTCDateTime(2023, 4, 30, 14, 4, 56, 216953","UTCDateTime(2023, 4, 30, 14, 4, 56, 332783",139.373515,2,NaN,2.220940,0.715173,False
4,3.449043,3.569163,3300,5145,0.12012,"UTCDateTime(2023, 4, 30, 20, 3, 25, 449043","UTCDateTime(2023, 4, 30, 20, 3, 25, 569163",141.043846,2,NaN,2.316993,0.770769,False
5,0.411723,0.514683,897,2232,0.10296,"UTCDateTime(2023, 4, 30, 20, 5, 22, 411723","UTCDateTime(2023, 4, 30, 20, 5, 22, 514683",140.981905,2,NaN,3.169667,0.812613,False
